In [ ]:
!pip install contractions
!pip install Unidecode
!pip install Pattern

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 48.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 9

In [ ]:
import pandas as pd
import re 
import unidecode
import re 
import time
import string
import statistics

from datetime import datetime

import contractions

In [ ]:
def sampleEqually(data_name, sample=0.1, lines=False, text_cleaner=None):
  data = pd.read_json("drive/MyDrive/" + data_name, lines = lines)
  if(text_cleaner is not None):
    data["review_text"] = data["review_text"].apply(text_cleaner)
  ## Let's calculate 10% of the data 
  TOTAL = len(data)
  N_SAMPLES = int(TOTAL * sample)
  print("Total:", TOTAL)
  print(N_SAMPLES)

  ## Calcualte number of false and true samples 
  false_samples = data[data["is_spoiler"] == False]
  true_samples = data[data["is_spoiler"] == True]

  N_TRUE = len(true_samples)
  N_FALSE = len(false_samples)
  print("No of false:", N_FALSE)
  print("No of true:", N_TRUE)
  print("% True", N_TRUE/ TOTAL * 100)
  print("% False", N_FALSE/ TOTAL * 100)

  ## Get half true and half false 

  n_false_samples = false_samples.sample(int(N_SAMPLES / 2))
  n_true_samples = true_samples.sample(int(N_SAMPLES / 2) +1 )

  N_FALSE_FILTERED = len(n_false_samples)
  N_TRUE_FILTERED = len(n_true_samples)
  print("No of false:", N_FALSE_FILTERED)
  print("No of true:", N_TRUE_FILTERED)
  print("% True", N_TRUE_FILTERED/ TOTAL * 100)
  print("% False", N_FALSE_FILTERED/ TOTAL * 100)

  FILTERED_TOTAL = N_TRUE_FILTERED + N_FALSE_FILTERED
  ## Selected 10% 
  print(FILTERED_TOTAL)
  print(N_SAMPLES)

  filtered_data = pd.concat([n_false_samples, n_true_samples], axis=0)

  filtered_data = filtered_data.sample(frac=1)

  return filtered_data

## Cleaning functions

In [ ]:
import nltk 
from nltk.corpus import stopwords # Stopwords 
from nltk.stem.porter import PorterStemmer
from pattern.text.en import singularize
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def remove_links(text):
    """
    This function will remove links from the 
    text contained within the Dataset.
       
    arguments:
        input_text: "text" of type "String". 
                    
    return:
        value: "text" with removed links. 
    """
    
    text = re.sub(r'http\S+', '', text)
    return text

In [ ]:
def remove_special_characters(text):
  text = re.sub(r'[^\w\s]', '', text)

  return text

In [ ]:
## https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
## https://medium.com/analytics-vidhya/removing-stop-words-with-nltk-library-in-python-f33f53556cc1

def remove_stopwords(text):
  tokens = word_tokenize(text)
  english_stopwords = stop_words = set(stopwords.words('english'))
  tokens_wo_stopwords = [t for t in tokens if t not in english_stopwords]
  text = " ".join(tokens_wo_stopwords)

  return text

In [ ]:
def remove_double_spaces(text):
  text = re.sub(r'\s{2,}', ' ', text)

  return text

In [ ]:
# Expand contraction words
def expand_contractions(text):
    """
    This Function will expands contractions in words.
    
    arguments:
         input_text: "text" of type "String".
         
    return:
        value: "text" with expanded contractions .  
    """
    
    # creating an empty list
    expanded_words = []    
    for word in text.split():
      # using contractions.fix to expand the shotened words
      expanded_words.append(contractions.fix(word)) 
    
    String_Of_tokens = ' '.join(expanded_words)
    
    return String_Of_tokens

In [ ]:
# Remove accented characters
def accented_characters_removal(text):
    """
    This function will remove accented characters from the 
    text contained within the Dataset.
       
    arguments:
    
        input_text: "text" of type "String". 
                    
    return:
        value: "text" with removed accented characters.  
    """
    
    # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
    text = unidecode.unidecode(text) 
    
    return text

In [ ]:
def little_process(text):
    
    text = str(text)
    
    text = remove_links(text)

    text = remove_double_spaces(text)


    return text

In [ ]:
def double_extra_cleaning(text):
  text = str(text)

  text = text.lower()

  text = remove_links(text)

  text = remove_special_characters(text)

  text = remove_double_spaces(text)

  text = remove_stopwords(text)

  return text

In [ ]:
def extra_cleaning(text):
    text = str(text)

    text = text.lower()
    
    text = remove_links(text)
    
    text = remove_special_characters(text)

    text = remove_double_spaces(text)

    return text

In [ ]:
def special_cleaning(text):
    text = text.lower()
    
    text = remove_links(text)
    
    text = expand_contractions(text)
    
    text = accented_characters_removal(text)
    
    text = remove_special_characters(text)

    return text